In [1]:
import pandas as pd
import os
import numpy as np
import glob
from pathlib import Path
from datetime import datetime as dt
from datetime import timedelta
import pyodbc
import shutil

In [2]:
home = Path.home()
today = dt.today().date()

dateRange = [today - timedelta(days = x) for x in range(365)]
#dateRange = [i.strftime("%Y%m%d") for i in dateRange]

today = today.strftime("%Y%m%d")


ODMdict = {
    'FWH' : 'WHFXN',
    'FXN' : 'WHFXN',
    'Foxconn WH' : 'WHFXN',
    'Compal' : 'KSCEI',
    'CEI' : 'KSCEI',
    'Wistron' : 'CQWIS',
    'wistron' : 'CQWIS',
    'Wistron CQ': 'CQWCQ',
    'WCQ': 'CQWIS',
    'Inventec' : 'CQIEC',
    'Inventec CQ' : 'CQIEC',
    'Quanta' : 'CQQCI',
    'Quanta CQ' : 'CQQCI',
    'Pegatron' : 'CQPCQ'
}

In [3]:
dateRange.reverse()

In [4]:
def clean(fname: str, file : pd.DataFrame, externalReportDate : str) -> pd.DataFrame:

    currentYear = dt.now().year
    currentday = fname.split('\\')[-1][-13:-5]
    file = file.assign(LastSGreportDate = currentday)
    
    file['LastSGreportDate'] = file['LastSGreportDate'].apply(lambda x: dt.strptime(x, '%Y%m%d'))
    file['LastSGreportDate'] = pd.to_datetime(file['LastSGreportDate'])

    file = file.assign(reportDate = externalReportDate)

    file['reportDate'] = file['reportDate'].apply(lambda x: dt.strptime(x, '%Y%m%d'))
    file['reportDate'] = pd.to_datetime(file['reportDate'])

    #clean
    file.columns = file.columns.str.strip()

    #drop useless columns and rows
    file = file.drop(columns = ['Description (Item)', 'Schedule (Comments)', 'Hub inventory', 'Vendor'])
    file = file[(file['Procurement type'] == 'B/S') | (file['Procurement type'] == 'Buysell') | (file['Procurement type'] == 'BS')].reset_index(drop = True)

    #adjust qty columns name and units
    qtycol = file.filter(like='Single Shortage QTY (K)').columns.tolist()
    

    for i in qtycol:
        file[i] = file[i].apply(lambda x: x.upper().strip() if type(x) == str else x)
        file[i] = file[i].replace("NEW", 0)
        file[i] = file[i].replace("NEW ADD", 0)
        file[i] = file[i].apply(lambda x: x*1000)
    file = file.rename(columns= {qtycol[0]: 'Single Shortage QTY', qtycol[1]: 'Prev_Single Shortage QTY'})

    #replace ODM name
    file['ODM'] = file['ODM'].replace(ODMdict)
    return file   

In [5]:
path = os.path.join(home, 'HP Inc','GPS TW Innovation - Documents','Project team','Project RiXin - Shortage management')
target = Path (path,'Single shortage')
PNFVPath = Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PN FV description mapping table_ALL.xlsx')
ExternalReportFolder = Path(path, 'External_Destination', 'today')

PNFVFile = pd.read_excel(PNFVPath)
PNFVFile = PNFVFile [['PN', 'Descr']]
PNFVFile = PNFVFile.rename(columns = {'PN': 'HP PN'})

In [6]:
fileList = [str(x) for x in Path(target,'Archive').glob("*xlsx")]
fileListDateNum = [dt.strptime(x[-13:-5], "%Y%m%d").date() for x in fileList]

errorList = []
resultList = []

In [7]:
lookupSGdateList = []
i = 0
for _ in dateRange:
    if i == len(fileListDateNum)-1:
        lookupSGdateList.append(fileListDateNum[i])
        continue
    
    if _ < fileListDateNum[i+1]:
        lookupSGdateList.append(fileListDateNum[i])
    else:
        i = i + 1
        lookupSGdateList.append(fileListDateNum[i])

In [8]:
dateRange = [i.strftime("%Y%m%d") for i in dateRange]
lookupSGdateList = [i.strftime("%Y%m%d") for i in lookupSGdateList]

In [9]:
zip(dateRange, lookupSGdateList)

dateList = result['reportDate'].tolist()
max(dateList)

In [10]:
def addKey(res: pd.DataFrame) -> tuple[list, pd.DataFrame]:
    LatestSGMaterial = res
    LatestSGMaterial = LatestSGMaterial.merge(PNFVFile, on = 'HP PN', how = 'left')
    LatestSGMaterial['Key'] = LatestSGMaterial['ODM'] + LatestSGMaterial['Descr']
    KeyList = LatestSGMaterial['Key'].tolist()
    print("addKey done!")
    return KeyList, res

### concat current day external report

In [11]:
def concatExternal(extReportPathList: list, KeyList: list) -> pd.DataFrame:
    externalResultDFList = []
    if not extReportPathList:
        return 

    for _ in extReportPathList:
        try: 
            temp = pd.read_excel(_)
            temp['ODM'] = temp['ODM'].ffill()
            temp['ODM'] = temp['ODM'].replace(ODMdict)
            temp['FV/Des'] = temp['FV/Des'].ffill()
            #temp['ETA'] = temp['ETA'].ffill()
            temp['key'] = temp['ODM'] + temp['FV/Des']
            temp = temp[temp.key.isin(KeyList)]
            
            try:
                temp = temp[['ODM', 'FV/Des', 'HP_PN', 'ETA', 'GPS Remark']]
            except:
                temp = temp[['ODM', 'FV/Des', 'HP PN', 'ETA', 'GPS Remark']]
                temp = temp.rename(columns = {'HP PN' : 'HP_PN'})
                print("Rocky wrong format!")
                
            temp = temp.groupby(['ODM', 'FV/Des']).agg({'ETA' : lambda x: '\n'.join(set(x.dropna())),
                                                        'GPS Remark': lambda x: '\n'.join(set(x.dropna()))})
            temp = temp.reset_index()
            if len(temp) > 0:
                print(len(temp))
                externalResultDFList.append(temp)
            else:
                pass
        except Exception as e:
            print(e)
            print(_)
    try:
        externalResultDF = pd.concat(externalResultDFList)
    except Exception as e:
        print(e)
        print("No single shortage match!")
        return pd.DataFrame(columns = ['ODM', 'FV/Des', 'HP_PN', 'ETA', 'GPS Remark'])
    print('External process done!')
    return externalResultDF

### lookup PNFV and merge external reportm
### output

In [12]:
def mergeNoutput(SGres: pd.DataFrame, extRes: pd.DataFrame, dateStr: str) -> None:
    SGres = SGres.merge(PNFVFile.rename(columns = {'PN': 'HP PN'}), on = 'HP PN', how = 'left')
    print(SGres[SGres['HP PN'] == 'M83662-N13'])
    SGres = SGres.merge(extRes.rename(columns = {'FV/Des' : 'Descr'}), on = ['ODM', 'Descr'], how = 'left')
    SGres = SGres.drop_duplicates()
    SGres.to_excel(Path(target, 'test','total singal shortage_' + dateStr +'.xlsx'), index = False)
    print("Output done!")
    return SGres

In [13]:
dateRange.reverse()

In [14]:
lookupSGdateList.reverse()

In [15]:
for i, j in zip(dateRange[0], lookupSGdateList[0]):
    print(i, j)

2 2
0 0
2 2
3 3
0 0
8 7
0 1
2 9


In [16]:
[f for f in glob.glob(str(Path(target, 'Single shortage ' + '*')))][-1]

'C:\\Users\\hsudu\\HP Inc\\GPS TW Innovation - Documents\\Project team\\Project RiXin - Shortage management\\Single shortage\\Single shortage 20230802.xlsx'

In [17]:
et = today
# et = '20230712'
# sg = '20230209'

fname = [f for f in glob.glob(str(Path(target, 'Single shortage ' + '*')))][-1]
ExternalReport = [f for f in glob.glob(str(Path(ExternalReportFolder, et + '*')))]
if not ExternalReport:
    print("No external on " + et)
    #exit()

try:
    file = pd.read_excel(str(fname))
    sg_res = clean(str(fname), file, et)
    print(str(fname) + " process done!")
except Exception as e:
    errorList.append([str(fname), e])
    print(e)
    print(str(fname) + " process failed!")
    #exit()

k, sg_res = addKey(sg_res)
ext_res = concatExternal(ExternalReport, k)
if ext_res is None:
    print("No external on " + et)
    #exit()

sg_res = mergeNoutput(sg_res, ext_res, et)

C:\Users\hsudu\HP Inc\GPS TW Innovation - Documents\Project team\Project RiXin - Shortage management\Single shortage\Single shortage 20230802.xlsx process done!
addKey done!
1
1
1
1
1
1
1
External process done!
Empty DataFrame
Columns: [Commodity, Single Shortage QTY, ODM, Series, HP PN, Prev_Single Shortage QTY, Category, Procurement type, Remark, CPU family, LastSGreportDate, reportDate, Descr]
Index: []
Output done!


In [18]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

In [19]:
#sg_res = sg_res.rename(columns={'HP PN' : 'HP_PN'})

In [20]:
sg_res['ETA'] = sg_res['ETA'].apply(lambda x: x.replace("'","") if type(x) == str else x)

In [21]:
sg_res

,Commodity,Single Shortage QTY,ODM,Series,HP PN,Prev_Single Shortage QTY,Category,Procurement type,Remark,CPU family,LastSGreportDate,reportDate,Descr,ETA,GPS Remark
0,CPU,6182.0,WHFXN,BPC,L92234-001,5315.0,BDT,B/S,WatsonIDG,Comet Lake,2023-08-02,2023-08-02,DT Intel Comet Lake i3-10100,ETA 08/04 * 1312 \nETA 08/07 * 1791(1463 (WWSB...,
1,CPU,46.0,WHFXN,WS,L54065-001,0.0,BDT,B/S,Sierra,CASCADELAKE,2023-08-02,2023-08-02,WS INTEL XEON CASCADELAKE-SP AU 5222,ETA 08/25*46 pcs/Intel/L54065-001 (WWSB from AMS),
2,GPU,1000.0,CQIEC,800s,N05293-001,0.0,BNB,B/S,ASAP Pull-in needed,NaN,2023-08-02,2023-08-02,QN20-P3-R-A1,"\nPer DSM, by 08/26 * 1340, by 08/26 * 550",
3,Camera,1200.0,CQIEC,1000S,M53214-171,700.0,BNB,B/S,ASAP Pull-in needed,NaN,2023-08-02,2023-08-02,2F NMc MIPI-RAW 5MP IR SD NFOV - F5NRS,ETA 08/02*163 pcs/TFC/M53214-971\nETA 08/03*54...,
4,LCD,1900.0,CQIEC,800s,L75543-N33,900.0,BNB,B/S,ASAP Pull-in needed,NaN,2023-08-02,2023-08-02,13.3FHD AG UWVA EDP NB BENT 250N,ETA 08/04*100 pcs/IVO/L75543-N33 (pull in fr 8...,
5,LCD,7700.0,CQIEC,OPP,M12358-3G1,0.0,CNB,B/S,ASAP Pull-in needed,NaN,2023-08-02,2023-08-02,17.3HD+ AG SVA EDP SLIMNBZ 250N,"ETA 08/10*2,217 pcs/BOE/M12358-LG1 (BT)\nETA 0...",
6,LCD,4500.0,CQQCI,S600,L35365-331,0.0,CNB,B/S,NaN,NaN,2023-08-02,2023-08-02,13.3FHD AG UWVA EDPPSR USLMNB LP 400N,ETA 08/04*11 pcs/LGD/L35365-235 (pull in 11 fr...,


In [22]:
for index, row in sg_res.iterrows():
    sg_Commodity = row['Commodity']
    sg_Qty = row['Single Shortage QTY']
    sg_ODM = row['ODM']
    sg_series = row['Series']
    sg_PN_all = row['HP PN']
    sg_pQty = row['Prev_Single Shortage QTY']
    sg_pTyoe = row['Procurement type']
    sg_reportDate = row['reportDate']
    sg_ETA = row['ETA']
    sg_gpsRemark = row['GPS Remark']
    sg_lastSGreportDate = row['LastSGreportDate']
    #sg_PN_single = row['HP PN']


    cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_Single_shortage ( Commodity, [Single Shortage QTY], ODM, Series, [HP PN], [Prev_Single Shortage QTY], \
                    [Procurement type], reportDate, ETA, [GPS Remark], LastSGreportDate)\
                    VALUES('{sg_Commodity}','{sg_Qty}','{sg_ODM}','{sg_series}','{sg_PN_all}', '{sg_pQty}','{sg_pTyoe}','{sg_reportDate}',\
                           '{sg_ETA}','{sg_gpsRemark}', '{sg_lastSGreportDate}')".replace("'NaT'", "NULL").replace("'nan'", "NULL"))

conn.commit()
conn.close()

In [24]:
ExternalreportArchiveFolder = os.path.join(path, 'External_Destination', 'Archive')

for f in os.listdir(ExternalReportFolder):
    if f.endswith('.xlsx'):
        shutil.move(os.path.join(ExternalReportFolder, f), os.path.join(ExternalreportArchiveFolder, f))

for f in os.listdir(os.path.join(ExternalReportFolder, 'amend')):
    if f.endswith('.xlsx'):
        shutil.move(os.path.join(ExternalReportFolder, 'amend', f), os.path.join(ExternalreportArchiveFolder, f))
